In [1]:
import warnings
warnings.filterwarnings("ignore")

import yfinance as yf
import pandas as pd
import vectorbt as vbt
import pytz

# ==============================
# CONFIG
# ==============================
symbol = "^NSEI"          # Example: NIFTY index
interval = "1m"
period = "7d"
profit_target = 0.0003   # 0.03%
timezone = pytz.timezone("Asia/Kolkata")

# ==============================
# DOWNLOAD DATA
# ==============================
data = yf.download(
    tickers=symbol,
    interval=interval,
    period=period,
    progress=False
)

# Convert timezone to IST
data.index = data.index.tz_convert(timezone)

# Keep only market hours (9:15 to 3:30)
data = data.between_time("09:15", "15:30")

close = data['Close']


In [2]:

# ==============================
# INDICATORS
# ==============================
ema7 = vbt.MA.run(close, window=7, ewm=True).ma
ema21 = vbt.MA.run(close, window=21, ewm=True).ma

# ==============================
# ENTRY SIGNAL
# ==============================
entries = ema7.vbt.crossed_above(ema21)

# ==============================
# EXIT AT 3:15 PM
# ==============================
exit_time = close.index.time >= pd.to_datetime("15:15").time()
time_exit = pd.Series(exit_time, index=close.index)

# ==============================
# PORTFOLIO
# ==============================
close = data["Close"].squeeze()

pf = vbt.Portfolio.from_signals(
    close=close,
    entries=entries,
    exits=time_exit,
    tp_stop=profit_target,
    sl_stop=profit_target*2, 
    direction="longonly",
    freq="1T",
    init_cash=100000,
    # fees=0.0002,
    # slippage=0.0001
)



In [3]:
pf.stats()

Start                         2026-01-27 09:15:00+05:30
End                           2026-02-04 15:29:00+05:30
Period                                  1 days 13:30:00
Start Value                                    100000.0
End Value                                 100542.417329
Total Return [%]                               0.542417
Benchmark Return [%]                           2.486553
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                               0.332175
Max Drawdown Duration                   0 days 17:03:00
Total Trades                                       58.0
Total Closed Trades                                58.0
Total Open Trades                                   0.0
Open Trade PnL                                      0.0
Win Rate [%]                                  67.241379
Best Trade [%]                                 0.189131
Worst Trade [%]                               -0

In [4]:
assert isinstance(close, pd.Series)

In [5]:
entries.index == close.index
time_exit.index == close.index

array([ True,  True,  True, ...,  True,  True,  True], shape=(2250,))

In [6]:
close.ndim == 1

True

In [7]:
import os
print(os.getcwd())

/Users/nook/Desktop/Backtest/Nifty


In [8]:
trades_df = pf.trades.records_readable.copy()

for col in trades_df.columns:
    if pd.api.types.is_datetime64tz_dtype(trades_df[col]):
        trades_df[col] = trades_df[col].dt.tz_localize(None)

# trades_df.to_excel("nifty_ema_trades_without_fees_with_SL.xlsx", index=False)
trades_df.to_excel("Testing.xlsx", index=False)
